# 2 - Workflow management with atomate

In this lesson we cover a few more advanced features of the atomate software package.  More specifically we'll go over an example that includes parents and children, talk more about where you can find preset workflows, and cover a basic example of analyzing workflow-generated materials data.

## The elastic tensor: A multi-step preset workflow

In [19]:
from atomate.vasp.workflows.presets.core import wf_elastic_constant
from pymatgen import MPRester

In [20]:
mpr = MPRester()

In [21]:
structure = mpr.get_structure_by_material_id("mp-149", conventional_unit_cell=True)
# TODO: viewer here?
structure

Structure Summary
Lattice
    abc : 5.468727957193058 5.468727957193058 5.468727957193058
 angles : 90.0 90.0 90.0
 volume : 163.55316755397953
      A : 5.468727957193058 0.0 3.348630094092061e-16
      B : -3.348630094092061e-16 5.468727957193058 3.348630094092061e-16
      C : 0.0 0.0 5.468727957193058
PeriodicSite: Si (4.1015, 4.1015, 4.1015) [0.7500, 0.7500, 0.7500]
PeriodicSite: Si (2.7344, 0.0000, 0.0000) [0.5000, 0.0000, 0.0000]
PeriodicSite: Si (4.1015, 1.3672, 1.3672) [0.7500, 0.2500, 0.2500]
PeriodicSite: Si (2.7344, 2.7344, 2.7344) [0.5000, 0.5000, 0.5000]
PeriodicSite: Si (1.3672, 4.1015, 1.3672) [0.2500, 0.7500, 0.2500]
PeriodicSite: Si (0.0000, 0.0000, 2.7344) [0.0000, 0.0000, 0.5000]
PeriodicSite: Si (1.3672, 1.3672, 4.1015) [0.2500, 0.2500, 0.7500]
PeriodicSite: Si (-0.0000, 2.7344, 0.0000) [0.0000, 0.5000, 0.0000]

In [24]:
wf = wf_elastic_constant(structure)

In [26]:
# TODO: plotter?
[fw.name for fw in wf.fws]

['Si-elastic structure optimization',
 'Si-elastic deformation 0',
 'Si-elastic deformation 1',
 'Si-elastic deformation 2',
 'Si-elastic deformation 3',
 'Si-elastic deformation 4',
 'Si-elastic deformation 5',
 'Si-elastic deformation 6',
 'Si-elastic deformation 7',
 'Si-elastic deformation 8',
 'Si-elastic deformation 9',
 'Si-elastic deformation 10',
 'Si-elastic deformation 11',
 'Si-elastic deformation 12',
 'Si-elastic deformation 13',
 'Si-elastic deformation 14',
 'Si-elastic deformation 15',
 'Si-elastic deformation 16',
 'Si-elastic deformation 17',
 'Si-elastic deformation 18',
 'Si-elastic deformation 19',
 'Si-elastic deformation 20',
 'Si-elastic deformation 21',
 'Si-elastic deformation 22',
 'Si-elastic deformation 23',
 'Analyze Elastic Data']

From our exploration of the workflow, we can see that the workflow has a "root" firework, which is the structure optimization firework, and a single "leaf" firework, which is the analysis task.  In between, we have several fireworks which have "elastic deformation" in their titles, each of which represents a calculation which will transform the output of the optimization firework by applying a unique strain.

In the analysis step, the stresses from those deformations and the strains are aggregated and used to fit an elastic tensor.

To run the workflow, we import our launchpad and rocket launcher tools.  Since we can't use VASP in the workflow, we're also going to "fake" vasp by copying files rather than running the binary.

In [97]:
from fireworks import LaunchPad
from atomate.vasp.powerups import use_fake_vasp

In [99]:
# TODO: get this working
wf = use_fake_vasp(wf, ref_dirs="PUT_REF_DIRS_HERE")
lpad = LaunchPad()
lpad.reset('', require_password=False)
lpad.add_wf(wf)

AttributeError: 'str' object has no attribute 'keys'

In [94]:
from fireworks.core.rocket_launcher import launch_rocket, rapidfire

In [ ]:
launch_rocket()

## Powerups

**TODO: I'm not sure this is all that useful, so it might go on the choppping block first**

Atomate contains a number of tools for modifying workflows, known as "powerups" which are functions which will modify the workflows in commonly desired ways.  For example, if you want to modify the parameters of a VASP calculation, you can use the add_modify_incar powerup to change these for every firework in the workflow.

In [27]:
from atomate.vasp.powerups import add_modify_incar

In [35]:
[task for task in wf.fws[0].tasks]

[FileWriteTask(None,
               {'files_to_write': [{'contents': '',
                  'filename': 'FW--Si-elastic_structure_optimization'}]}),
 WriteVaspFromIOSet(None, {'structure': Structure Summary
                     Lattice
                         abc : 5.468727957193058 5.468727957193058 5.468727957193058
                      angles : 90.0 90.0 90.0
                      volume : 163.55316755397953
                           A : 5.468727957193058 0.0 3.348630094092061e-16
                           B : -3.348630094092061e-16 5.468727957193058 3.348630094092061e-16
                           C : 0.0 0.0 5.468727957193058
                     PeriodicSite: Si (4.1015, 4.1015, 4.1015) [0.7500, 0.7500, 0.7500]
                     PeriodicSite: Si (2.7344, 0.0000, 0.0000) [0.5000, 0.0000, 0.0000]
                     PeriodicSite: Si (4.1015, 1.3672, 1.3672) [0.7500, 0.2500, 0.2500]
                     PeriodicSite: Si (2.7344, 2.7344, 2.7344) [0.5000, 0.5000, 0.5000]
      

In [36]:
modified = add_modify_incar(wf, {"incar_update": {"ENCUT": 700}})

In [38]:
modified.fws[0].tasks

[FileWriteTask(None,
               {'files_to_write': [{'contents': '',
                  'filename': 'FW--Si-elastic_structure_optimization'}]}),
 WriteVaspFromIOSet(None, {'structure': Structure Summary
                     Lattice
                         abc : 5.468727957193058 5.468727957193058 5.468727957193058
                      angles : 90.0 90.0 90.0
                      volume : 163.55316755397953
                           A : 5.468727957193058 0.0 3.348630094092061e-16
                           B : -3.348630094092061e-16 5.468727957193058 3.348630094092061e-16
                           C : 0.0 0.0 5.468727957193058
                     PeriodicSite: Si (4.1015, 4.1015, 4.1015) [0.7500, 0.7500, 0.7500]
                     PeriodicSite: Si (2.7344, 0.0000, 0.0000) [0.5000, 0.0000, 0.0000]
                     PeriodicSite: Si (4.1015, 1.3672, 1.3672) [0.7500, 0.2500, 0.2500]
                     PeriodicSite: Si (2.7344, 2.7344, 2.7344) [0.5000, 0.5000, 0.5000]
      

Prior to the execution of the VASP calculation, a ModifyIncar firetask is inserted which will read the incar and modify the ENCUT parameter such that it matches our powerup specification.

## A few other "complex" workflows.

Atomate contains a few workflows that will do more complicating things like calculations on surfaces.  Let's say I want to calculate the determine the hydrogen adsorption energy on every low-index facet of Pt.

In [44]:
from atomate.vasp.workflows.base.adsorption import get_wf_surface_all_slabs
from pymatgen import MPRester, Molecule

In [45]:
pt = mpr.get_structure_by_material_id("Pt", conventional_unit_cell=True)
adsorbate = Molecule("H", [[0, 0, 0]])

In [47]:
wf = get_wf_surface_all_slabs(pt, [adsorbate])

In [50]:
[fw.name for fw in wf.fws]

['Pt-structure optimization',
 'Pt-Pt_(1, 1, 1) slab optimization',
 'Pt-H1-Pt_(1, 1, 1) adsorbate optimization 0',
 'Pt-H1-Pt_(1, 1, 1) adsorbate optimization 1',
 'Pt-H1-Pt_(1, 1, 1) adsorbate optimization 2',
 'Pt-H1-Pt_(1, 1, 1) adsorbate optimization 3',
 'Pt-Pt_(1, 1, 0) slab optimization',
 'Pt-H1-Pt_(1, 1, 0) adsorbate optimization 0',
 'Pt-H1-Pt_(1, 1, 0) adsorbate optimization 1',
 'Pt-H1-Pt_(1, 1, 0) adsorbate optimization 2',
 'Pt-H1-Pt_(1, 1, 0) adsorbate optimization 3',
 'Pt-Pt_(1, 0, 0) slab optimization',
 'Pt-H1-Pt_(1, 0, 0) adsorbate optimization 0',
 'Pt-H1-Pt_(1, 0, 0) adsorbate optimization 1',
 'Pt-H1-Pt_(1, 0, 0) adsorbate optimization 2']

This is how you can do the requisite calculation to find the band-gap from an HSE calculation after the PBE bandstructure.

In [63]:
from atomate.vasp.workflows.presets.core import wf_bandstructure_plus_hse

In [66]:
wf = wf_bandstructure_plus_hse(structure)

In [67]:
[fw.name for fw in wf.fws]

['Si-structure optimization',
 'Si-static',
 'Si-nscf uniform',
 'Si-nscf line',
 'Si-hse gap']

There's also support for workflows in FEFF and QChem.

In [72]:
from atomate.feff.workflows.core import get_wf_xas

In [78]:
wf = get_wf_xas("Si", structure)
[fw.name for fw in wf.fws]

['Si-XANES-K-0']

In [80]:
# TODO: double check that this will actually work
from atomate.qchem.workflows.base import double_FF_opt

ModuleNotFoundError: No module named 'pymatgen.io.qchem_io'

## Analyzing workflow results

**TODO: flesh out description**, use stores?

Our final example for atomate will take a set of tasks from our database and construct a phase diagram

In [ ]:
chemsys = ["Li", "Fe", "P", "O"]

